# EMBC 2021 – Saliva Analysis

Saliva Analysis for the Paper "Assessing the Influence of the Inner Clock on the Cortisol Awakening Response and Pre-Awakening Movement" (EMBC 2021)

## Setup

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from biopsykit.utils.dataframe_handling import multi_xs
from biopsykit.stats import StatsPipeline
from biopsykit.protocols import CAR

from carwatch_analysis.saliva_helper import import_cortisol_raw, import_cortisol_features, analysis_saliva_raw, analysis_saliva_features
from carwatch_analysis.general_helper import describe_groups_df

from statannot import add_stat_annotation

import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
pg.options['round'] = 3

palette = bp.colors.fau_palette
sns.set_theme(context='talk', style='ticks', palette=palette)
#plt.rcParams['figure.figsize'] = (15,5)
plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['pdf.fonttype'] = 42
plt.close('all')

export = True
palette

In [ ]:
export_path = Path("../../../exports")
plot_path = export_path.joinpath("plots")
stats_path = export_path.joinpath("statistics")

paper_path = Path("/Users/Richer/Documents/PhD/Research/Papers/2021_CARInnerClock_EMBC/img")
#paper_path = paper_path.joinpath('img')

bp.utils.file_handling.mkdirs([plot_path, stats_path, paper_path])

## Data Import

In [ ]:
car = CAR(saliva_times=[i*15 for i in range(5)])

### Raw Cortisol

In [ ]:
cort_samples = import_cortisol_raw(export_path.joinpath("cortisol_samples_cleaned.csv"))
cort_samples.head()

### Cortsol Features

In [ ]:
cort_features = import_cortisol_features(export_path.joinpath("cortisol_features_cleaned.csv"))
cort_features.head()

## Plots and Statistics

In [ ]:
condition_order = ['Spontaneous', 'Known Alarm', 'Unknown Alarm']

### Class Distribution

In [ ]:
describe_groups_df(cort_samples, "condition", order=condition_order)

### Raw Cortisol

#### Plots

In [ ]:
x = 'sample'
y = 'cortisol'
hue = 'condition'
style = 'condition'

sns.set_theme(context='talk', style='ticks', palette=palette)
fig, ax = plt.subplots(figsize=(10,4))
car.car_plot(data=cort_samples, x=x, y=y, hue=hue, order=condition_order, style=style, ax=ax)

fig.tight_layout()
if export:
    [fig.savefig(p.joinpath("img_car_raw_condition.pdf"), transparent=True) for p in [plot_path, paper_path]]

#### Statistics

In [ ]:
pipeline = analysis_saliva_raw(cort_samples, variable="condition")
pipeline.display_results(posthoc=False)
if export:
    pipeline.export_statistics(stats_path.joinpath("analysis_cortisol_raw_condition.xlsx"))

### Cortisol Features

#### Statistics

In [ ]:
pipeline = analysis_saliva_features(cort_features, variable="condition", test_type='anova')
pipeline.display_results(posthoc=True, sig_only="posthoc")
if export:
    pipeline.export_statistics(stats_path.joinpath("analysis_cortisol_features_condition.xlsx"))

#### Result Table for Paper

In [ ]:
index_labels = {
    'auc_g': r"$AUC_G$",
    'auc_i': r"$AUC_I$",
    'slopeS0S3': r"$a_{S0S3}$",
    'slopeS0S4': r"$a_{S0S4}$",
    'max_inc': "$\Delta c_{max}$",
    'cini': "$c_{ini}$",
    'cmax': "$c_{max}$"
}
pipeline.df_to_latex('anova', index_labels)

#### Plots

In [ ]:
features = ['auc', 'slopeS0S3', 'max_inc']
hue = 'condition'

box_pairs, pvals = pipeline.sig_brackets(pipeline.results_cat('posthoc'), stats_type='between', plot_type='multi')

sns.set_theme(context='talk', style='ticks', palette=palette, font_scale=1.2)
fig, axs = plt.subplots(figsize=(15,5), ncols=len(features))
car.saliva_multi_feature_boxplot(data=cort_features, saliva_type='cortisol', hue=hue, features=features, hue_order=condition_order, stats_kwargs={'box_pairs': box_pairs, 'pvalues': pvals}, ax=axs, rect=(0, 0, 0.81, 1.0), legend_fontsize='small')

if export:
    [fig.savefig(p.joinpath("img_boxplots_car_condition.pdf"), transparent=True) for p in [plot_path, paper_path]]

## TODOs

**TODOs:**
* (later) move all condition stuff to `Questionnaire_Processing.ipynb` and only import index in `Saliva_Proessing.ipynb`